In [ ]:
import carto2gpd
import cenpy
import numpy as np
import pandas as pd
import dask.dataframe as dd
import geopandas as gpd
import datetime as dt
import missingno as msno
import hvplot.pandas
import holoviews as hv

from io import BytesIO
import gzip
from urllib.request import urlopen
from zipfile import ZipFile


import math 
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt


In [ ]:
states = ['pa','il','ga','ca']
years = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

In [ ]:
url = "https://lehd.ces.census.gov/data/lodes/LODES7/pa/wac/pa_wac_S000_JT00_"
wac = dd.concat([pd.read_csv(gzip.open(BytesIO(urlopen(url+year+".csv.gz").read()))).assign(year=year) for year in years])


variable description
![Drag](WAX-data-description-2.png)

In [ ]:
wac['w_geocode'] = wac['w_geocode'].astype('str').str[:12]
wac.compute()

In [ ]:
# Block group download link: https://www2.census.gov/geo/tiger/TIGER2019/BG/tl_2019_42_bg.zip
phl_bg = gpd.read_file("tl_2019_42_bg/tl_2019_42_bg.shp").query("COUNTYFP in ['017','029','045','091','101']").to_crs('EPSG:3857')

In [ ]:
# unzip the file and find the .shp file we need
resp = urlopen("https://www2.census.gov/geo/tiger/TIGER2019/BG/tl_2019_42_bg.zip")
zipfile = ZipFile(BytesIO(resp.read()))
filenames = zipfile.namelist()

total number analysis

In [ ]:
wac_geo = phl_bg[['GEOID','geometry']].merge(wac.compute(), left_on='GEOID', right_on='w_geocode', how='inner')
wac_all = wac_geo.groupby('GEOID')['C000'].sum().reset_index()

In [ ]:
wac_all1 = pd.merge(wac_geo[["GEOID","geometry"]],wac_all,on="GEOID",how="inner")
wac_all1 = wac_all1.drop_duplicates()

In [ ]:
wac_all1["log_c000"] = wac_all1["C000"].apply(lambda x: math.log10(x))

In [ ]:
wac_all1.hvplot(geo=True,
                c="log_c000",
                crs=3857,
                hover_fill_color="gray",
                width=600, 
                height=400,
                cmap='RdBu_r')

In [ ]:
wac_all1.hvplot(geo=True,
                c="C000",
                crs=3857,
                hover_fill_color="gray",
                width=600, 
                height=400,
                cmap='RdBu_r')

Cluster

In [ ]:
wac_all2 = wac_all1[["C000","GEOID","geometry"]]

In [ ]:
# Scaler expects the features to be a 2D array with shape: (number of observations, number of features). 
# We are explicitly adding a second axis with the np.newaxis variable.

scaler = StandardScaler()
scaled_features = scaler.fit_transform(wac_all2["C000"][:, np.newaxis])

scaled_features

In [ ]:
# Elbow method to find k

# # Number of clusters to try out
n_clusters = list(range(2, 10))

# Run kmeans for each value of k
inertias = []
for k in n_clusters:
    
    # Initialize and run
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(scaled_features)
    
    # Save the "inertia"
    inertias.append(kmeans.inertia_)
    
# Plot it!
plt.plot(n_clusters, inertias, marker='o', ms=10, mfc='white', lw=4, mew=3)

The number of clusters seems to be 4.

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=8)

# Run the fit!
kmeans.fit(scaled_features)

# Save the cluster labels
wac_all2['label'] = kmeans.labels_

In [ ]:
# Calculate average features per cluster
wac_all2.groupby('label', as_index=False).size()



In [ ]:
wac_all2.head()

In [ ]:
# plot the data
wac_all2 = wac_all2.to_crs(epsg=3857)

# setup the figure
f, ax = plt.subplots(figsize=(10, 8))

# plot, coloring by label column
# specify categorical data and add legend
wac_all2.plot(
    column="label",
    cmap="Dark2",
    categorical=True,
    legend=True,
    edgecolor="k",
    lw=0.5,
    ax=ax,
)


ax.set_axis_off()
plt.axis("equal");

In [ ]:
wac_all2["label"] = wac_all2["label"].astype('str')

In [ ]:
wac_all2.hvplot(geo=True,
                c="label",
                crs=3857,
                hover_fill_color="gray",
                width=600, 
                height=400,
                cmap='RdBu_r')

# DBSCAN

In [ ]:
from sklearn.cluster import dbscan 

In [ ]:
# some parameters to start with
eps = 50  # in meters
min_samples = 50

cores, labels = dbscan(wac_all2["C000"][:, np.newaxis], eps=eps, min_samples=min_samples)

In [ ]:
wac_all2['label_dbscan'] = labels

In [ ]:
num_cores = len(cores)
print(f"Number of core samples = {num_cores}")

In [ ]:
# The number of clusters is the number of unique labels minus one (because noise has a label of -1)

num_clusters = wac_all2['label_dbscan'].nunique() - 1
print(f"number of clusters = {num_clusters}")

In [ ]:
cluster_sizes = wac_all2.groupby('label_dbscan', as_index=False).size()

cluster_sizes

Number of cluster = 4. Number of noise point is 1530.

In [ ]:
# plot the data
wac_all2 = wac_all2.to_crs(epsg=3857)

# setup the figure
f, ax = plt.subplots(figsize=(10, 8))

# plot, coloring by label column
# specify categorical data and add legend
wac_all2.plot(
    column="label_dbscan",
    cmap="Dark2",
    categorical=True,
    legend=True,
    edgecolor="k",
    lw=0.5,
    ax=ax,
)


ax.set_axis_off()
plt.axis("equal");

In [ ]:
wac_all2["label_dbscan"] = wac_all2["label_dbscan"].astype("str")

In [ ]:
wac_all2.hvplot(geo=True,
                c="label_dbscan",
                crs=3857,
                hover_fill_color="gray",
                width=600, 
                height=400,
                cmap='RdBu_r')

In [ ]:
# analyse more variables
variables = ['GEOID', 'geometry', 'w_geocode', 'C000', 'CA01', 'CA02', 'CA03',
       'CE01', 'CE02', 'CE03', 'CNS01', 'CNS02', 'CNS03', 'CNS04', 'CNS05',
       'CNS06', 'CNS07', 'CNS08', 'CNS09', 'CNS10', 'CNS11', 'CNS12', 'CNS13',
       'CNS14', 'CNS15', 'CNS16', 'CNS17', 'CNS18', 'CNS19', 'CNS20', 'CR01',
       'CR02', 'CR03', 'CR04', 'CR05', 'CR07', 'CT01', 'CT02', 'CD01', 'CD02',
       'CD03', 'CD04', 'CS01', 'CS02','year']
wac_var = wac_geo[variables]

In [ ]:
# Elbow method to find k

scaler = StandardScaler()
scaled_features = scaler.fit_transform(wac_var.drop(['year','GEOID','geometry'],axis=1))

scaled_features
# # Number of clusters to try out
n_clusters = list(range(2, 10))

# Run kmeans for each value of k
inertias = []
for k in n_clusters:
    
    # Initialize and run
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(scaled_features)
    
    # Save the "inertia"
    inertias.append(kmeans.inertia_)
    
# Plot it!
plt.plot(n_clusters, inertias, marker='o', ms=10, mfc='white', lw=4, mew=3)

In [ ]:
# some parameters to start with
eps = 50  # in meters
min_samples = 50

cores, labels = dbscan(wac_var.drop(['year','GEOID','geometry'],axis=1), eps=eps, min_samples=min_samples)

In [ ]:
wac_var['label_dbscan'] = labels

In [ ]:
cluster_sizes = wac_var.groupby('label_dbscan', as_index=False).size()

cluster_sizes

In [ ]:
wac_var["label_dbscan",'year','geometry'].hvplot(geo=True,
                c="label_dbscan",
                crs=3857,
                hover_fill_color="gray",
                width=1000, 
                height=850,
                cmap='RdBu_r')

In [ ]:
wac_var